In [ ]:
import datetime
from sgp4.api import Satrec, WGS72
from sgp4.api import jday
import pyproj
import concurrent.futures

def read_tle_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    tles = [(lines[i], lines[i+1], lines[i+2]) for i in range(0, len(lines), 3)]
    return tles

def propagate_satellite(tle, start_time, end_time, interval_minutes=1):
    satellite = Satrec.twoline2rv(tle[1], tle[2], WGS72)
    jd_start, fr_start = jday(start_time.year, start_time.month, start_time.day,
                              start_time.hour, start_time.minute, start_time.second)

    jd_end, fr_end = jday(end_time.year, end_time.month, end_time.day,
                          end_time.hour, end_time.minute, end_time.second)

    num_intervals = int((jd_end - jd_start) * 1440 / interval_minutes) + 1
    results = []

    for i in range(num_intervals):
        jd = jd_start + i * interval_minutes / 1440
        fr = fr_start
        e, r, v = satellite.sgp4(jd, fr)
        if e == 0:
            results.append((jd, r, v))

    return results

def ecef2lla(pos_x, pos_y, pos_z):
    ecef = pyproj.Proj(proj="geocent", ellps="WGS84", datum="WGS84")
    lla = pyproj.Proj(proj="latlong", ellps="WGS84", datum="WGS84")
    lon, lat, alt = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
    return lon, lat, alt

def filter_positions_by_region(positions, region):
    filtered_positions = []
    for jd, lon, lat, alt in positions:
        if (region['lat_min'] <= lat <= region['lat_max'] and
            region['lon_min'] <= lon <= region['lon_max']):
            filtered_positions.append((jd, lon, lat, alt))
    return filtered_positions

def process_satellite(tle):
    sat_name = tle[0].strip()
    positions = propagate_satellite(tle, start_time, end_time)
    converted_positions = [
        (jd, *ecef2lla(r[0], r[1], r[2])) for jd, r, v in positions
    ]
    filtered_positions = filter_positions_by_region(converted_positions, region)
    return sat_name, filtered_positions


tle_file_path = '30sats.txt'
tles = read_tle_file(tle_file_path)

start_time = datetime.datetime(2024, 1, 1, 0, 0, 0)
end_time = datetime.datetime(2024, 1, 2, 0, 0, 0)


region = {
    'lat_min': 16.66673, 'lat_max': 69.74973,
    'lon_min': -120.64459, 'lon_max': 103.58196
}

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(process_satellite, tles))

optimized_filtered_positions = {sat_name: positions for sat_name, positions in results}

print(optimized_filtered_positions)


Streaming output truncated to the last 5000 lines.
<ipython-input-5-7b6e6ca5ecc3>:36: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat, alt = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
<ipython-input-5-7b6e6ca5ecc3>:36: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat, alt = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
<ipython-input-5-7b6e6ca5ecc3>:36: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat, alt = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
<ipython-input-5-7b6e6ca5ecc3>:36: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat,

{'ISS (ZARYA)': [], 'AEROCUBE 12A': [], 'AEROCUBE 12B': [], 'ICS-EF (ISS DEB)': [], 'ISS DEB': [], 'MMSATS-1': [], 'CSS (TIANHE)': [], 'ISS (NAUKA)': [], 'FREGAT DEB': [], 'KITSUNE': [], 'CSS (WENTIAN)': [], 'HSU-SAT1': [], 'D3': [], 'BEAVERCUBE': [], 'CLICK-A': [], 'SOYUZ-MS 22': [], 'CREW DRAGON 5': [], 'CSS (MENGTIAN)': [], 'CYGNUS NG-18': [], 'TIANZHOU-5': [], 'SHENZHOU-15': [], '1998-067UL': [], '1998-067UM': [], '1998-067UN': [], '1998-067UP': [], 'XW-4 (CAS-10)': [], 'MARIO': [], 'NUTSAT': [], 'LORIS': [], 'ORCASAT': [], 'TJREVERB': [], 'SPORT': [], 'PETITSAT': [], 'SS-1': [], 'HSKSAT': [], 'OPTIMAL-1': [], 'ISS DEB (SPX-26 IPA FSE)': [], 'PROGRESS-MS 22': [], 'SOYUZ-MS 23': [], 'SL-4 R/B': []}


propagate_satellite function uses the SGP4 model to calculate the position (ECEF coordinates) and velocity of a satellite at one-minute intervals over a specified time range.
ecef2lla function converts ECEF (Earth-Centered, Earth-Fixed) coordinates to latitude, longitude, and altitude.
filter_positions_by_region function filters satellite positions to include only those that lie within a user-defined rectangular region specified by latitude and longitude bounds.
process_satellite function combines the propagation, conversion, and filtering steps for a single satellite. It returns the satellite name and its filtered positions